# Fragestellung 1.1: Wie gross ist der Anteil von Gas-, Heizöl- und Elektroheizungen in einer Gemeinde?

Um diese Frage zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. <br>
[Link zum GWR](https://www.housing-stat.ch/de/index.html)<br>
[Link zur GWR Dokumentation](https://www.housing-stat.ch/de/docs/index.html)<br>
[Link zum GWR Download](https://www.housing-stat.ch/de/madd/public.html)<br>
<br>
Die Daten werden nach Gemeinde und Energiequelle gruppiert und dann die einzelnen Objekte gezählt. <br>
Anschliessend wird die resultierende Liste als CSV-Datei zur weiterverarbeitung bzw. Visualisierung exportiert.<br>
<br>
---
<i> CAS Spatial Data Analytics 2022 </i> ¦ <i> Heizenergieträger in Gemeinden </i> ¦ <i> Stand: 21.11.2022  </i> ¦ <i> Entwickler: Jürg Reist </i>

### Notebook vorbereiten und die benötigten Daten aus dem GWR einlesen

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# Datenquelle definieren
url = urlopen("https://public.madd.bfs.admin.ch/ch.zip")

#lesen GWR Daten
zipfile = ZipFile(BytesIO(url.read()))
dfGWRSource = pd.read_csv(zipfile.open('gebaeude_batiment_edificio.csv'),
                         usecols=['GGDENAME', 'GKODE', 'GKODN', 'GENH1', 'GWAERSCEH1', 'GWAERDATH1'],
                         sep='\t')

dfGWRSource.head()

,GGDENAME,GKODE,GKODN,GENH1,GWAERSCEH1,GWAERDATH1
0,Aeugst am Albis,2679647.268,1237500.347,7530.0,860.0,2002-09-05
1,Aeugst am Albis,2680635.900,1236936.229,7530.0,860.0,2002-09-05
2,Aeugst am Albis,2679016.828,1235828.156,7530.0,860.0,2002-09-05
3,Aeugst am Albis,2679677.869,1237453.256,7530.0,869.0,2001-11-29
4,Aeugst am Albis,2679692.997,1237491.284,7530.0,869.0,2014-01-14


In [2]:
#Column mit geeigneten Namen
dfGWRSource = dfGWRSource.rename(columns={"GGDENAME": "Gemeinde",
                              "GKODE": "lat",
                              "GKODN": "lon",
                              "GENH1":"Energiequelle",
                              "GWAERSCEH1":"Quelle",
                              "GWAERDATH1": "Update"})

In [3]:
dfGWRSource

,Gemeinde,lat,lon,Energiequelle,Quelle,Update
0,Aeugst am Albis,2679647.268,1237500.347,7530.0,860.0,2002-09-05
1,Aeugst am Albis,2680635.900,1236936.229,7530.0,860.0,2002-09-05
2,Aeugst am Albis,2679016.828,1235828.156,7530.0,860.0,2002-09-05
3,Aeugst am Albis,2679677.869,1237453.256,7530.0,869.0,2001-11-29
4,Aeugst am Albis,2679692.997,1237491.284,7530.0,869.0,2014-01-14
...,...,...,...,...,...,...
3064792,La Baroche,2582768.637,1251359.261,7530.0,860.0,2001-11-29
3064793,La Baroche,2582743.072,1251406.135,7530.0,860.0,2001-11-29
3064794,La Baroche,2584600.501,1251495.804,7520.0,860.0,2001-11-29
3064795,La Baroche,2583393.948,1251448.443,7540.0,860.0,2001-11-29


In [7]:
# Die Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
# Um die finale Visualisierung auf die drei definierten Energiequellen (Gas, Heizöl, Elektrizität) zu fokussieren, werden die Energiequellen zusammengefasst.
# Im GWR-Datenfile sind folgende Energiequellen definiert:
# 7501: 'Luft', 7510: 'Erdwärme (generisch', 7511: 'Erdwärmesonde', 7512: 'Erdregister', 7513: 'Wasser (Grundwasser, Oberflächenwasser, Abwasser)'
# 7520: 'Gas', 7530: 'Heizöl', 7540: 'Holz (generisch)', 7541: 'Holz (Stückholz)', 7542: 'Holz (Pellets)', 7543: 'Holz (Schnitzel)', 7550: 'Abwärme (innerhalb des Gebäudes)'
# 7560: 'Elektrizität', 7570: 'Sonne (thermisch)', 7580: 'Fernwärme (generisch)', 7581: 'Fernwärme (Hochtemperatur)', 7582: 'Fernwärme (Niedertemperatur)'
# 7598: 'Unbestimmt', 7599: 'Andere'

dfGWRSource['Energiequelle'] = dfGWRSource.Energiequelle.replace({
                                            7500: 'Keine',
                                            7501: 'Weitere',
                                            7510: 'Weitere',
                                            7511: 'Weitere',
                                            7512: 'Weitere',
                                            7513: 'Weitere',
                                            7520: 'Gas',
                                            7530: 'Heizöl',
                                            7540: 'Weitere',
                                            7541: 'Weitere',
                                            7542: 'Weitere',
                                            7543: 'Weitere',
                                            7550: 'Weitere',
                                            7560: 'Elektrizität',
                                            7570: 'Weitere',
                                            7580: 'Weitere',
                                            7581: 'Weitere',
                                            7582: 'Weitere',
                                            7598: 'Unbestimmt',
                                            7599: 'Weitere'
                                            })

In [8]:
#Dort wor keine Energiequelle (NULL) angegeben ist, wird der Text "Unbestimmt" gesetzt
dfGWRSource["Energiequelle"].fillna("Unbestimmt", inplace = True)


In [9]:
# Die korrekten Bezeichnungen der Informationsquellen zuweisen und als Liste speichern.
dfGWRSource['Quelle'] = dfGWRSource.Quelle.replace({
                                            852: 'Gemäss amtliche Schätzung',
                                            853: 'Gemäss Gebäudeversicherung',
                                            855: 'Gemäss Kontrolle der Heizungseinrichtungen',
                                            857: 'Gemäss Eigentümer/in / Verwaltung',
                                            858: 'Gemäss Gebäudeenergieausweis der Kantone (GEAK)',
                                            859: 'Andere Informationsquelle',
                                            860: 'Gemäss Volkszählung 2000',
                                            864: 'Gemäss Daten des Kantons',
                                            865: 'Gemäss Daten der Gemeinde',
                                            869: 'Gemäss Baubewilligung',
                                            870: 'Gemäss Versorgungswerk (Gas, Fernwärme)',
                                            871: 'Gemäss Minergie'
                                            })

In [10]:
dfGWRSource

,Gemeinde,lat,lon,Energiequelle,Quelle,Update
0,Aeugst am Albis,2679647.268,1237500.347,Heizöl,Gemäss Volkszählung 2000,2002-09-05
1,Aeugst am Albis,2680635.900,1236936.229,Heizöl,Gemäss Volkszählung 2000,2002-09-05
2,Aeugst am Albis,2679016.828,1235828.156,Heizöl,Gemäss Volkszählung 2000,2002-09-05
3,Aeugst am Albis,2679677.869,1237453.256,Heizöl,Gemäss Baubewilligung,2001-11-29
4,Aeugst am Albis,2679692.997,1237491.284,Heizöl,Gemäss Baubewilligung,2014-01-14
...,...,...,...,...,...,...
3062575,La Baroche,2582768.637,1251359.261,Heizöl,Gemäss Volkszählung 2000,2001-11-29
3062576,La Baroche,2582743.072,1251406.135,Heizöl,Gemäss Volkszählung 2000,2001-11-29
3062577,La Baroche,2584600.501,1251495.804,Gas,Gemäss Volkszählung 2000,2001-11-29
3062578,La Baroche,2583393.948,1251448.443,Weitere,Gemäss Volkszählung 2000,2001-11-29


### Nur die Attribute in ein neues Dataframe speichern, die ausgewertet werden sollen

In [11]:
#Datenframe für Verteilung der Energietrager pro Gemeinde
dfEnergyProGemeinde = dfGWRSource[['Gemeinde', 'Energiequelle']]
dfEnergyProGemeinde

,Gemeinde,Energiequelle
0,Aeugst am Albis,Heizöl
1,Aeugst am Albis,Heizöl
2,Aeugst am Albis,Heizöl
3,Aeugst am Albis,Heizöl
4,Aeugst am Albis,Heizöl
...,...,...
3062575,La Baroche,Heizöl
3062576,La Baroche,Heizöl
3062577,La Baroche,Gas
3062578,La Baroche,Weitere


### Zählen aller Gebäude pro Gemeinde und deren Energiequelle inkl. Nullwerte

In [12]:
#Zählt alle Gebäude pro Gemeinde und deren Energiequelle inkl. Nullwerte
dfEnergyProGemeinde = dfEnergyProGemeinde[['Gemeinde', 'Energiequelle']].value_counts(dropna=False).reset_index()
dfEnergyProGemeinde

,Gemeinde,Energiequelle,0
0,Zürich,Gas,21642
1,Zürich,Keine,14192
2,Zürich,Heizöl,12709
3,Zürich,Weitere,12028
4,Winterthur,Unbestimmt,10899
...,...,...,...
12345,Eptingen,Gas,1
12346,Roggenburg,Gas,1
12347,Romont (BE),Gas,1
12348,Engelberg,Gas,1


### Spalten benennen und Daten in ein CSV-File exportieren
Diese CSV ist dann die Grundlage für das Dashboard

In [13]:
#Spaltenaen neu definieren
dfEnergyProGemeinde.columns  = ['Gemeinde', 'Energiequelle','Anzahl']
dfEnergyProGemeinde

,Gemeinde,Energiequelle,Anzahl
0,Zürich,Gas,21642
1,Zürich,Keine,14192
2,Zürich,Heizöl,12709
3,Zürich,Weitere,12028
4,Winterthur,Unbestimmt,10899
...,...,...,...
12345,Eptingen,Gas,1
12346,Roggenburg,Gas,1
12347,Romont (BE),Gas,1
12348,Engelberg,Gas,1


In [14]:
#Als CSV-File exportieren
dfEnergyProGemeinde.to_csv('Daten/Gemeindeliste_1-1.csv',sep=',', encoding="utf-8-sig")